# Reading and Visualizing GeoJSON Data

In [ ]:
!pip install folium

In [28]:
import geopandas as gpd
import folium
from IPython.display import display, clear_output
from ipywidgets import interact_manual
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, widgets

In [29]:
# Read the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file("../data/neighborhoods/nycneighborhoods.geojson.json")
unique_boroughs = gdf['borough'].unique()

## Visualize the map

In [44]:
# Create a dropdown widget
borough_dropdown = widgets.Dropdown(
    options=unique_boroughs,
    value='Manhattan',  # Default value
    description='Borough:',
)


def update_map(borough):
    # Clear the previous map
    clear_output(wait=True)
    
    display(borough_dropdown)
    
    
    # Filter the GeoDataFrame by the selected borough
    gdf_filtered = gdf[gdf['borough'] == borough]
    
    # Calculate the centroid of the filtered GeoDataFrame to center the map
    x_filtered_center = gdf_filtered.geometry.unary_union.centroid.x
    y_filtered_center = gdf_filtered.geometry.unary_union.centroid.y
    
    # Create a new folium map centered around the calculated coordinates
    m = folium.Map(location=[y_filtered_center, x_filtered_center], zoom_start=13)
    
        # Add borders around each neighborhood
    folium.GeoJson(
        gdf_filtered,
        style_function=lambda feature: {
            'color': 'black',
            'weight': 1,
        }
    ).add_to(m)

    
    # Iterate over the filtered GeoDataFrame to add markers
    for idx, row in gdf_filtered.iterrows():
        x = row['geometry'].centroid.x
        y = row['geometry'].centroid.y
        neighborhood_name = row['neighborhood']
        popup_text = f"{neighborhood_name}, {borough}"
        popup = folium.Popup(popup_text, parse_html=True)
        folium.Marker([y, x], popup=popup).add_to(m)
        
        # Add labels
        label = folium.map.Marker(
            [y, x],
            icon=folium.DivIcon(
                icon_size=(150,36),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 10pt; color : black">{neighborhood_name}</div>',
                )
            )
        label.add_to(m)
        
        
    # Show the map
    display(m)

# Link the dropdown widget to the update_map function
borough_dropdown.observe(lambda change: update_map(change['new']), names='value')

# Display the dropdown widget
display(borough_dropdown)

# Display the default map
update_map('Manhattan')


Dropdown(description='Borough:', index=4, options=('Bronx', 'Queens', 'Staten Island', 'Brooklyn', 'Manhattan'…

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

gdf.boundary.plot(ax=ax)

# Get the coordinates to center the map
x_map_center = gdf.geometry.unary_union.centroid.x
y_map_center = gdf.geometry.unary_union.centroid.y

# Create a folium map
m = folium.Map(location=[y_map_center, x_map_center], zoom_start=13)

folium.GeoJson(gdf).add_to(m)

# Iterate over the GeoDataFrame to get each polygon's (neighborhood's) details
for idx, row in gdf.iterrows():
    # Get the x and y coordinates of the polygon's centroid
    x = row['geometry'].centroid.x
    y = row['geometry'].centroid.y
    
    # Assuming the column that contains the neighborhood name is called 'Name'
    neighborhood_name = row['neighborhood']
    borough_name = row['borough']
    
    # Create a folium Popup object, with the neighborhood name
    popup_text = f"{neighborhood_name}, {borough_name}"
    popup = folium.Popup(popup_text, parse_html=True)
    
    # Create a folium Marker at the centroid of the polygon, with the popup
    folium.Marker([y, x], popup=popup).add_to(m)

m
